In [7]:
from hypergraphs import *

In [8]:
carbohydrateMetabolismPathways = ["00010","00020","00030","00040","00051","00052","00053","00500","00520","00620","00630","00640","00650","00660","00562"]

In [9]:
H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBMLs(["../../LaTeX/Data/hsa/hsa{}-sbml.xml".format(x) for x in carbohydrateMetabolismPathways])

Ignoring file ../../LaTeX/Data/hsa/hsa00660-sbml.xml, couldn't get a model out of it


In [10]:
dd=hyper_cardon_crochemore(H)

In [11]:
showMapKeys(dd, onlyNonTrivial=True, keyadj=sr2r, prefOnly="r", keyfamily=r2ss)

{'rnR01639', 'rnR01529'}
{'rnR00238', 'rnR01975', 'rnR00410'}
{'rnR04779', 'rnR03321', 'rnR01819', 'rnR02740', 'rnR00768'}
{'rnR01051', 'rnR01049', 'rnR01056', 'rnR01057'}
{'rnR03469', 'rnR05795', 'rnR11680'}
{'rnR04513', 'rnR09827'}
{'rnR05800', 'rnR03433', 'rnR03394'}
{'rnR00658', 'rnR00200'}
{'rnR01518', 'rnR01512', 'rnR01516'}
{'rnR01104', 'rnR01329'}
{'rnR00658', 'rnR01518'}
{'rnR01051', 'rnR01049', 'rnR01057'}
{'rnR00289', 'rnR00959', 'rnR01660', 'rnR08639'}
{'rnR00955', 'rnR00503', 'rnR00291'}
{'rnR01104', 'rnR01329'}
{'rnR02750', 'rnR02749'}
{'rnR00754', 'rnR00746'}
{'rnR00760', 'rnR00866', 'rnR00875', 'rnR00867'}
{'rnR01392', 'rnR01221', 'rnR01394', 'rnR00945'}
{'rnR01639', 'rnR01896'}
{'rnR08639', 'rnR00771', 'rnR00303', 'rnR07324'}
{'rnR01526', 'rnR01056', 'rnR01529'}
{'rnR00405', 'rnR00432', 'rnR00833'}
{'rnR02705', 'rnR01804', 'rnR01207'}
{'rnR04435', 'rnR01805'}
{'rnR02640', 'rnR02933'}
{'rnR01070', 'rnR04780'}
{'rnR00316', 'rnR00710', 'rnR00711', 'rnR00235'}
{'rnR08193',

In [12]:
dd.keys()

dict_keys(['C00084', 'C00033', 'C00469', 'C00068', 'C00022', 'C05125', 'C15972', 'C16255', 'C15973', 'C00024', 'C00186', 'C00074', 'C00631', 'C00197', 'C00118', 'C00236', 'C00111', 'C05378', 'C05345', 'C00668', 'C00103', 'C01172', 'C00221', 'C00267', 'C01159', 'C00026', 'C05381', 'C16254', 'C00091', 'C00042', 'C05379', 'C00311', 'C00036', 'C00158', 'C00122', 'C00149', 'C00417', 'C01801', 'C00673', 'C00672', 'C00117', 'C00119', 'C00620', 'C00121', 'C05382', 'C00231', 'C00199', 'C00279', 'C00345', 'C01236', 'C00257', 'C00198', 'C00258', 'C03033', 'C00191', 'C00309', 'C01068', 'C00789', 'C00310', 'C00379', 'C00312', 'C00181', 'C00800', 'C00618', 'C00029', 'C00167', 'C02266', 'C00794', 'C00095', 'C01094', 'C00159', 'C00275', 'C00665', 'C00636', 'C02985', 'C00325', 'C01222', 'C00096', 'C00577', 'C01019', 'C01720', 'C03827', 'C01613', 'C00124', 'C00492', 'C05404', 'C05402', 'C00089', 'C00031', 'C05401', 'C00116', 'C05400', 'C05399', 'C01235', 'C00137', 'C00243', 'C00052', 'C00446', 'C00984',

In [13]:
showMapKeys(dd, onlyNonTrivial=True, keydesc=mid2mname, prefOnly="C")

{'C04063 (C04063)', 'C04477 (C04477)'}
{'C11555 (C11555)', 'C01220 (C01220)'}
{'C04520 (C04520)', 'C01204 (C01204)'}
{'C00026 (C00026)', 'C05379 (C05379)'}
{'C03033 (C03033)', 'C00190 (C00190)'}
{'C00641 (C00641)', 'C05981 (C05981)'}
{'C00119 (C00119)', 'C00121 (C00121)', 'C00620 (C00620)'}
{'C00168 (C00168)', 'C00037 (C00037)'}
{'C00014 (C00014)', 'C00258 (C00258)', 'C00065 (C00065)', 'C01146 (C01146)'}
{'C00672 (C00672)', 'C01801 (C01801)'}
{'C00181 (C00181)', 'C00312 (C00312)'}
{'C18060 (C18060)', 'C00334 (C00334)'}
{'C00232 (C00232)', 'C00203 (C00203)'}
{'C00999 (C00999)', 'C05402 (C05402)', 'C00064 (C00064)', 'C01177 (C01177)', 'C01674 (C01674)', 'C20238 (C20238)', 'C21384 (C21384)', 'C00257 (C00257)', 'C03827 (C03827)', 'C00492 (C00492)', 'C00007 (C00007)', 'C00058 (C00058)', 'C00185 (C00185)', 'C00136 (C00136)'}
{'C00198 (C00198)', 'C00227 (C00227)', 'C00665 (C00665)', 'C01935 (C01935)', 'C01083 (C01083)', 'C06002 (C06002)', 'C01898 (C01898)', 'C01720 (C01720)', 'C05404 (C05404)

In [14]:
classSizeDist(dd, prefOnly="r", keyadj=sr2r)

{1: 66, 2: 44, 3: 28, 5: 5, 4: 14, 7: 1, 28: 1}

In [15]:
classSizeDist(dd, prefOnly="C")

{1: 126, 2: 15, 3: 1, 4: 1, 14: 1, 26: 1, 10: 1}